In [1]:
import tensorflow as tf
print("Using TensorFlow version %s" % tf.__version__)

Using TensorFlow version 2.5.0


In [2]:
import numpy as np

##12.2 넘파이처럼 텐서플로 사용하기

###12.2.1 텐서와 연산

In [3]:
tf.constant([[1., 2., 3.],
             [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
# tf.convert_to_tensor
_array_with_reshape = np.arange(1, 7, dtype=np.float32).reshape((2, -1))
tf.convert_to_tensor(_array_with_reshape)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
_array = np.arange(1, 7, dtype=np.float32)
# tf.convert_to_tensor
_tf_array = tf.convert_to_tensor(_array)
# tf.reshape
_tf_reshaped = tf.reshape(_tf_array, (2, 3))
_tf_reshaped

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [6]:
tf.experimental.numpy?

In [7]:
# dtype, shape
_tf_reshaped.dtype, _tf_reshaped.shape

(tf.float32, TensorShape([2, 3]))

In [8]:
# indexing
_tf_reshaped[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [9]:
# tf.newaxis
_tf_reshaped[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

###12.2.2 텐서와 넘파이

In [10]:
# numpy array to tensor
a = np.array([2, 4, 5], dtype=np.float64)
t = tf.constant(a)
t

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [11]:
# tensor to numpy array
t.numpy()

array([2., 4., 5.])

###12.2.3 타입 변환

In [12]:
t.dtype

tf.float64

In [13]:
# tf.cast?

In [14]:
t = tf.cast(t, tf.float32) # inplace=False
t.dtype

tf.float32

###12.2.4 변수

In [15]:
_arr = np.arange(1, 7, dtype=np.float32).reshape((2, -1))
v = tf.Variable(_arr, name='test_variable')
v

<tf.Variable 'test_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [16]:
v * 2
v

<tf.Variable 'test_variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [17]:
# assign ; inplace=True
v.assign(v * 2)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [18]:
# indexing하여 assign
# broadcasting❌
v[:, 2].assign([77, 77])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4., 77.],
       [ 8., 10., 77.]], dtype=float32)>

###12.2.5 다른 데이터 구조


* 희소 텐서
* 텐서 배열
* 래그드 텐서
* 문자열 텐서
* 집합
* 큐

##12.3 사용자 정의 모델과 훈련 알고리즘

###12.3.1 사용자 정의 손실 함수

In [19]:
tf.keras.losses.Huber?

In [20]:
# tf.where?

In [21]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - .5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [22]:
# huber function standalone usage
np.random.seed(777)
y_true = np.random.randint(0, 5, 10)
y_true = y_true.astype(np.float64)
error = np.random.rand(10) * 2
y_pred = y_true + error
huber_fn(tf.constant(y_true), tf.constant(y_pred))

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([0.01269968, 0.67922751, 0.23576545, 1.4777523 , 0.75294641,
       0.86355856, 0.60451363, 0.14457146, 0.27864514, 0.09939387])>

In [23]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - threshold**2*0.5
        return tf.where(is_small_error, squared_loss, linear_loss)

In [24]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = tf.abs(error) - self.threshold**2*0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

In [25]:
hloss = HuberLoss(name='hloss')

In [26]:
hloss.get_config()

{'name': 'hloss', 'reduction': 'auto', 'threshold': 1.0}

In [27]:
def func(a, b, c):
    return a + b * c

In [28]:
abc_params = {'a':1, 'b':2, 'c':3}

In [29]:
func(**abc_params)

7

###12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

In [30]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [31]:
_weights = np.random.randint(1,10, 12).reshape([2, 2, 3])
_weights

array([[[8, 9, 1],
        [9, 4, 3]],

       [[1, 4, 4],
        [5, 1, 7]]])

In [32]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"facotr":self.factor}

In [33]:
my_l1_regularizer = MyL1Regularizer(2.)
my_l1_regularizer.__call__(_weights)

<tf.Tensor: shape=(), dtype=float64, numpy=112.0>

In [34]:
my_l1_regularizer.get_config()

{'facotr': 2.0}

###12.3.4 사용자 정의 지표

In [35]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1., **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

In [36]:
tf.keras.metrics.Metric.add_weight?

###12.3.5 사용자 정의 층

In [37]:
# 가중치가 없는 레이어
exp_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))
exp_layer(tf.constant(0.))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [38]:
tf.keras.activations.get('sigmoid')

<function tensorflow.python.keras.activations.sigmoid>

In [39]:
# 가중치를 갖는 레이어: MyDense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
    
    def call(self, X):
        return self.activation(X @ self.kernerl + self.bias) 
                                # @; matmul

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
                              # input_shape                       # output_shape
    
    def get_config(self):
        base_config = super().__init__(**kwargs)
        return {**base_config, 
                "units":self.units,
                "activation":tf.keras.activations.serialize(self.activation)}
                                                  # 활성함수 전체설정 저장

In [40]:
tf.keras.layers.Layer.add_weight?

In [41]:
tf.constant(np.arange(1, 7).reshape((3, 2))).shape.as_list()[:-1]+[10]
                                            # input_shape        # output_shape

[3, 10]

In [42]:
tf.keras.layers.GaussianNoise

tensorflow.python.keras.layers.noise.GaussianNoise